# Deciding Depot Location

This notebook contains the example code provided in the assignment text, with each task separated into its own cell.

Once you have written the appropriate code for a particular task, you should be able to run the corresponding cell to reproduce the example output.

This notebook assumes that:

- Your code (`country.py` and `utilities.py`) files are located in the same directory as this notebook.
- The `locations.csv` data file that we provided can be found at the relative path `./data/locations.csv` from the directory that this notebook is stored in.

If neither of these are correct, you will need to adjust the setup cell with the correct paths.

Remember that subsequent cells are dependent on cells that came before, so you will need to run the cells in this notebook **in sequence** from top to bottom. You can also use the "restart and run all" option to run the entire notebook from the beginning, in order - though you will get errors in those cells you haven't written the code for yet.

In [ ]:
# If you have moved the locations.csv file, you will need to update this variable
from pathlib import Path
locations_csv_file = Path("./data/locations.csv").resolve()

import numpy as np

default_speed = 4.75
# Travelling this distance without any penalties at the default speed should take 1 hour.
one_hour_distance = 3_600 * default_speed

## A Function for Travel Time

In [ ]:
from country import travel_time

print(f"Travelling at the default speed: {travel_time(one_hour_distance, 0., 3.):2.1f} h")
print(f"Travelling at half default speed: {travel_time(one_hour_distance, 0., 0., speed = default_speed / 2.):2.1f} h")
print(f"Travel at default speed but with a region penalty that equates to 3:  {travel_time(one_hour_distance, 1., 20.):2.1f} h")

## The `Location` Class

In [ ]:
from country import Location

riverwood = Location("Riverwood", "Whiterun Hold", 49_877.15654485528,-1.1153081421843865, False)
heartwood_mill = Location("Heartwood Mill", "The Rift", 164_031.25924652288,-0.6236682227787959, True)

In [ ]:
print(f"Before changing depot: {heartwood_mill.settlement}")
heartwood_mill.depot = False
print(f"After changing depot: {heartwood_mill.settlement}")

bad_name = Location("noT CAPitalised", "Region", 0.0, 0.0, False)
print(f"After creation, name is now: {bad_name.name}")

In [ ]:
# Attempting to set the settlement property directly should throw an error.
try:
    heartwood_mill.settlement = False
    print("If you are seeing this in the output, no error was raised!")
except Exception as e:
    print("Attempting to assign to the settlement property raised the following error:")
    print(f"\t{e}")

In [ ]:
riverwood = Location("Riverwood", "Whiterun Hold", 49_877.15654485528,-1.1153081421843865, False)
heartwood_mill = Location("Heartwood Mill", "The Rift", 164_031.25924652288,-0.6236682227787959, True)

print(f"Riverwood's information: {riverwood}")
print(f"Heartwood Mill's information: {heartwood_mill}")

In [ ]:
RW_to_HM = riverwood.distance_to(heartwood_mill)
print(f"Distance from Riverwood to Heartwood Mill: {RW_to_HM}")

In [ ]:
print(riverwood == riverwood)
print(riverwood == heartwood_mill)

# Do not forget that this task requires you to write a section in your report.

## The `Country` Class

In [ ]:
from country import Country

riverwood = Location("Riverwood", "Whiterun Hold", 49_877.15654485528,-1.1153081421843865, False)
heartwood_mill = Location("Heartwood Mill", "The Rift", 164_031.25924652288,-0.6236682227787959, True)
karthwasten = Location("Karthwasten", "The Reach", 138_231.89539682947,2.858973382047493, True)
whiterun = Location("Whiterun", "Whiterun Hold", 21_197.215713390284,-0.3577712724508101, False)
list_of_locations = [riverwood, heartwood_mill, karthwasten, whiterun]
country = Country(list_of_locations)

print("List of locations passed in:")
for loc in list_of_locations:
    print(f"\t{loc}")
assert isinstance(country._all_locations, tuple), "Locations are not stored as a tuple in the Country class"
assert set(country._all_locations) == set(list_of_locations), "Provided entries and those stored aren't the same"

In [ ]:
from utilities import read_country_data

skyrim = read_country_data(locations_csv_file)

In [ ]:
print(f"Number of settlements in Skyrim: {skyrim.n_settlements}")
print("List of those settlements:")
for settlement in skyrim.settlements:
  print(f"\t{settlement}")
print(f"Number of depots in Skyrim: {skyrim.n_depots}")
print("List of those depots:")
for depot in skyrim.depots:
  print(f"\t{depot}")

skyrim_map = skyrim.plot_country()
skyrim_map.show()

In [ ]:
RW_to_HM_time = skyrim.travel_time(riverwood, heartwood_mill)
HM_to_RW_time = skyrim.travel_time(heartwood_mill, riverwood)

print(f"Travel time from Riverwood to Heartwood Mill: {RW_to_HM_time}")
print(f"Travel time from Heartwood Mill to Riverwood: {HM_to_RW_time}")

kvatch = Location("Kvatch", "Cyrodiil", 175_000, -3. * np.pi / 4., False)
try:
  skyrim.travel_time(riverwood, kvatch)
  print("If you see this message in the output, no error was raised!")
except Exception as e:
  print("Attempting to determine travel time to a location not in the country threw an error:")
  print(f"\t{e}")

In [ ]:
# Of all the settlements in the country, find the one that has the shortest travel time from Riverwood
print(f"Using default args: {skyrim.fastest_trip_from(riverwood)}")
# Only consider the locations in skyrim.settlements[0, 1, 3, 4]
print(f"Selecting settlements: {skyrim.fastest_trip_from(riverwood, [0, 1, 3, 4])}")
# Only consider the locations provided explicitly
print(f"Providing explicit locations: {skyrim.fastest_trip_from(riverwood, [heartwood_mill, whiterun])}")
# Mix-and-match locations and indices
print(f"Mix and match selection: {skyrim.fastest_trip_from(riverwood, [0, whiterun, 2, 3, heartwood_mill])}")

In [ ]:
tour_from_heartwood_mill, tour_time = skyrim.nn_tour(heartwood_mill)

print(f"Time to complete tour starting in Heartwood Mill: {tour_time:2.2f} h")
print("The tour path was:")
for loc in tour_from_heartwood_mill:
  print(f"\t{loc}")

path_map = skyrim.plot_path(
    path=tour_from_heartwood_mill,
    polar_projection=True,
)
path_map.show()

In [ ]:
# This will produce no output text by itself
best_depot = skyrim.best_depot_site(display=False)

print("The best depot found was:", best_depot)

# This will automatically display some of the information that
# is computed before returning the best depot
print("\nWith display=True however, we get information automatically...\n")
best_depot_again = skyrim.best_depot_site()

assert best_depot_again == best_depot

## Execution Time for the NNA

Your script should be callable from the command line.
Running the cell below (with this notebook in the folder detailed at the start!) should also run your script as if it had been invoked with

```bash
python execution_time.py
```

In [ ]:
%run -i execution_time.py